In [ ]:
import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
prescriptions = pd.read_parquet("../20_intermediate_files/prescriptions_wa.parquet", engine = "fastparquet")

In [ ]:
prescriptions.columns

In [ ]:
# reducing the number of columns in the drug prescriptions dataset by including only the relevant attributes to create a new dataset

prescriptions_reduced = prescriptions[
    [
        "DRUG_CODE",
        "DRUG_NAME",
        "QUANTITY",
        "UNIT",
        "STRENGTH",
        "CALC_BASE_WT_IN_GM",
        "DOSAGE_UNIT",
        "Product_Name",
        "Ingredient_Name",
        "Measure",
        "MME_Conversion_Factor",
        "dos_str",
        "Year",
        "Month",
        "StateFIPS",
        "StateName",
        "CountyFIPS",
        "state_abbrev",
        "FIP_unique",
        "Population",
        "county_test",
    ]
]

In [ ]:
# creating a copy of reduced dataset of prescriptions and converting some of the attributes to appropriate data type

prescriptions_reduced_copy = prescriptions_reduced.copy()

prescriptions_reduced_copy["Year"] = prescriptions_reduced_copy["Year"].astype("int64")
prescriptions_reduced_copy["DRUG_CODE"] = prescriptions_reduced_copy[
    "DRUG_CODE"
].astype("int64")
prescriptions_reduced_copy["Month"] = prescriptions_reduced_copy["Month"].astype(
    "int64"
)
prescriptions_reduced_copy["Population"] = prescriptions_reduced_copy[
    "Population"
].astype("int64")

prescriptions_reduced_copy.rename(
    columns={
        "DRUG_CODE": "Drug Code",
        "DRUG_NAME": "Drug Name",
        "QUANTITY": "Quantity",
        "UNIT": "Unit",
        "STRENGTH": "Strength",
        "CALC_BASE_WT_IN_GM": "Calc Base Weight (In Gm)",
        "DOSAGE_UNIT": "Dosage Unit",
        "dos_str": "Dosage Strength",
        "StateFIPS": "State FIPS",
        "StateName": "State",
        "CountyFIPS": "County FIPS",
        "state_abbrev": "State Abbreviation",
        "FIP_unique": "FIPS_Unique",
        "county_test": "County",
    },
    inplace=True,
)

In [ ]:
prescriptions_reduced_copy.head(3)

In [ ]:
# creating a dataset that has all the drug prescriptions in the state of Washington

washington_prescriptions = prescriptions_reduced_copy[
    prescriptions_reduced_copy["State"] == "Washington"
]

washington_prescriptions.head()

In [ ]:
washington_prescriptions_copy = washington_prescriptions.copy()
washington_prescriptions_copy["shipment_per_100k"] = washington_prescriptions_copy["Quantity"] / washington_prescriptions_copy["Population"] * 100_000

In [ ]:
# washington_prescriptions_copy = washington_prescriptions.copy()

# washington_prescriptions_copy["shipment_per_100k"] = (
#     (
#         washington_prescriptions_copy["Dosage Strength"]
#         * washington_prescriptions_copy["Dosage Unit"]
#         * washington_prescriptions_copy["MME_Conversion_Factor"]
#     )
#     / (washington_prescriptions_copy["Population"])
#     * 100000
# )

# washington_prescriptions_copy.head(3)

In [ ]:
wa_result = washington_prescriptions_copy.groupby(["Year", "County"])["shipment_per_100k"].sum().reset_index()

In [ ]:
wa_stats = washington_prescriptions_copy.groupby(["Year"])["shipment_per_100k"].sum().reset_index()

In [ ]:
# wa_res = pd.DataFrame(wa_stats.describe()["overdose_per_100k"]).rename(columns={"overdose_per_100k": "Overdoses per 100k Residents - Washington"})
# comp_res = pd.DataFrame(comp_stats.describe()["overdose_per_100k"]).rename(columns={"overdose_per_100k": "Overdoses per 100k Residents - Comp States"})
# stats = pd.concat([wa_res, comp_res], axis=1)
# stats

In [ ]:
# create a scale for number of years before and after 2007 (target year)

def scale_years(year):
    if year == 2009:
        return -2
    if year == 2010:
        return -1
    if year == 2011:
        return 0
    if year == 2012:
        return 1
    # if year == 2013:
    #     return 1
    # if year == 2014:
    #     return 2
    # if year == 2015:
    #     return 3

wa_stats["year relative to policy"] = wa_stats["Year"].apply(lambda x: scale_years(x))
# comp_result["year relative to policy"] = comp_result["Year"].apply(lambda x: scale_years(x))

# double check no nulls in "year relative to policy"

assert (wa_stats["year relative to policy"].isnull().sum() == 0)
# assert (comp_result["year relative to policy"].isnull().sum() == 0)

# doing this in case the float == int comparison causes issues

In [ ]:
# split into before 2007 and after 2007

wa_b4 = wa_stats[wa_stats["Year"] <= 2011]
wa_after = wa_stats[wa_stats["Year"] >= 2011]


# wa_after = wa_after[wa_after["Year"] != 2007] # may need to handle this differently

In [ ]:
from sklearn.linear_model import LinearRegression 

regressor_b4 = LinearRegression() 
regressor_after = LinearRegression()


X_b4 = np.array(wa_b4["year relative to policy"]).reshape(-1, 1)
y_b4 = np.array(wa_b4["shipment_per_100k"]).reshape(-1, 1)

X_after = np.array(wa_after["year relative to policy"]).reshape(-1, 1)
y_after = np.array(wa_after["shipment_per_100k"]).reshape(-1, 1)


regressor_b4.fit(X_b4,y_b4) 
regressor_after.fit(X_after,y_after) 


y_pred_b4 = regressor_b4.predict(X_b4)
y_pred_after = regressor_after.predict(X_b4)

In [ ]:
y_pred_b4

In [ ]:
y_pred_after

In [ ]:
# plt.xlim(-3, 3)
# #plt.ylim(0, 500)

# plt.title("Opioid Shipments in Washington")

# plt.plot(X_b4, y_pred_b4,color='k')
# plt.plot(X_after, y_pred_after,color='k')

# # plot avg value in each year

In [ ]:
x = "Year"
y = "shipment_per_100k"

In [ ]:
def vertical_line(year):
    """Function to plot a vertical line at year of policy implementation"""
    line = alt.Chart(pd.DataFrame({
    'Date': [year],
    'color': ["black"]
    })).mark_rule().encode(
    x='Date:Q', # use q for "quantitative" - as per altair docs
    color=alt.Color('color:N', scale=None)
    )

    return line

In [ ]:
# test function
line = vertical_line(2004)

In [ ]:
def get_charts(b4, after, title_b4, title_after):
    """
    Function to plot the pre and post charts.
    Will not use in final plot - used as a baseline for our fit charts later.

    """

    base_before = (
        alt.Chart(b4)
        .mark_point()
        .encode(
            y=alt.Y("shipment_per_100k", scale=alt.Scale(zero=False)),
            x=alt.X("year relative to policy", scale=alt.Scale(zero=False)),
        )
        .properties(title=title_b4)
        
    )

    base_after = (
        alt.Chart(after)
        .mark_point()
        .encode(
            y=alt.Y("shipment_per_100k", scale=alt.Scale(zero=False)),
            x=alt.X("year relative to policy", scale=alt.Scale(zero=False)),
        )
        .properties(title=title_after)
    )

    return base_before, base_after

In [ ]:
# test the function
# may remove title parameters later - not really necessary as we aren't plotting this part in our final analysis
# however, if we can't add a title to our fit/regression line charts, we may need to add them here
base_before, base_after = get_charts(b4 = wa_b4, after = wa_after, title_b4 = "shipments before policy", title_after="shipments after policy")
base_before + base_after

In [ ]:
# starting here in final report
# no longer calculating base chart above - just adding in regression line at same time
def get_preds(df, x, y):

    # init new empty df for our predictions
    predictions = pd.DataFrame()

    # fit our model and predict values
    model = smf.ols(f"{y} ~ {x}", data=df).fit()
    model_predict = model.get_prediction(df[x])

    # save predictions back to df, calculate confidence intervals
    predictions["shipment_per_100k"] = model.predict(df[x])
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=0.05)

    # save original year columns to new predictions df
    predictions["Year"] = df["Year"]
    predictions["year relative to policy"] = df["year relative to policy"]
    return predictions

In [ ]:
wa_b4_preds = get_preds(wa_b4, x, y)
wa_after_preds = get_preds(wa_after, x, y)

In [ ]:
def get_charts(b4, after, title_b4, title_after, color):
    """
    Function to plot the pre and post charts.
    Will not use in final plot - used as a baseline for our fit charts later.

    """

    base_before = (
        alt.Chart(b4)
        .mark_point()
        .encode(
            y=alt.Y("shipment_per_100k", scale=alt.Scale(zero=False)),
            x=alt.X("year relative to policy", scale=alt.Scale(zero=False)),
        )
        .properties(title=title_b4).transform_regression("year relative to policy", "shipment_per_100k")
    .mark_line()
    .encode(color=alt.value(color))
        
    )

    base_after = (
        alt.Chart(after)
        .mark_point()
        .encode(
            y=alt.Y("shipment_per_100k", scale=alt.Scale(zero=False)),
            x=alt.X("year relative to policy", scale=alt.Scale(zero=False)),
            
        )
        .properties(title=title_after).transform_regression("year relative to policy", "shipment_per_100k")
    .mark_line()
    .encode(color=alt.value(color))
    )

    return base_before, base_after

In [ ]:
# test the function
# may remove title parameters later - not really necessary as we aren't plotting this part in our final analysis
# however, if we can't add a title to our fit/regression line charts, we may need to add them here
base_before, base_after = get_charts(b4 = wa_b4_preds, after = wa_after_preds, title_b4 = "Opioid Shipments Before and After Policy Implementation in Washington", title_after="Shipments After Policy Implementation", color="red")
base_before + base_after

In [ ]:
#####################################
# updated code for final report ends here 

In [ ]:
prescriptions["QUANTITY"].describe()

In [ ]:
prescriptions_reduced = prescriptions[['QUANTITY', 'Year', 'StateFIPS', 'StateName', 'CountyFIPS', 'FIP_unique', 'Population','county_test']]

In [ ]:
prescriptions_reduced["StateName"].value_counts()

In [ ]:
washington = prescriptions_reduced[prescriptions_reduced["StateName"] == "Washington"]
comp = prescriptions_reduced[prescriptions_reduced["StateName"] != "Washington"]

In [ ]:
wa_prescriptions = washington.copy()
comp_prescriptions = comp.copy()

In [ ]:
wa_prescriptions["quantity_sum"] = wa_prescriptions.groupby(["Year", "FIP_unique"])["QUANTITY"].transform('sum')
comp_prescriptions["quantity_sum"] = comp_prescriptions.groupby(["Year", "FIP_unique"])["QUANTITY"].transform('sum')

In [ ]:
wa_prescriptions = wa_prescriptions.drop('QUANTITY', axis=1)
comp_prescriptions = comp_prescriptions.drop('QUANTITY', axis=1)

In [ ]:
wa_result = wa_prescriptions.drop_duplicates()
comp_result = comp_prescriptions.drop_duplicates()

In [ ]:
wa_result["shipment_per_100k"] = wa_result["quantity_sum"] / wa_result["Population"] * 100_000
comp_result["shipment_per_100k"] = comp_result["quantity_sum"] / comp_result["Population"] * 100_000

In [ ]:
wa_result.groupby("Year")["shipment_per_100k"].agg([np.mean, np.std])

In [ ]:
wa_result_b4 = wa_result[wa_result["Year"] < 2012]
wa_result_after = wa_result[wa_result["Year"] >= 2012]

In [ ]:

# washington

source_data = wa_result_b4

plot_wa_b4 = alt.Chart(source_data).mark_point().encode(
    y=alt.Y("mean_shipment:Q", scale=alt.Scale(zero=False)),
    x=alt.X("Year:O", scale=alt.Scale(zero=False))
).transform_aggregate(
    mean_shipment='mean(shipment_per_100k)',
    groupby=["Year"]
)

plot_wa_b4

In [ ]:
fit_wa_b4 = plot_wa_b4.transform_regression('Year', 'mean_shipment',method="linear"
).mark_line()

fit_wa_b4

In [ ]:
source_data = wa_result_after

plot_wa_after = alt.Chart(source_data).mark_point().encode(
    y=alt.Y("mean_shipment:Q", scale=alt.Scale(zero=False)),
    x=alt.X("Year:O", scale=alt.Scale(zero=False))
).transform_aggregate(
    mean_shipment='mean(shipment_per_100k)',
    groupby=["Year"]
)

plot_wa_after

In [ ]:
fit_wa_after = plot_wa_after.transform_regression('Year', 'mean_overdose',method="linear"
).mark_line()

fit_wa_after

In [ ]:
plot_wa_b4 + fit_wa_b4 + plot_wa_after + fit_wa_after